In [ ]:
!mkdir -p price_of_books
!wget https://automl-mm-bench.s3.amazonaws.com/machine_hack_competitions/predict_the_price_of_books/Data.zip -O price_of_books/Data.zip
!cd price_of_books && unzip -o Data.zip
!ls price_of_books/Participants_Data

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import KFold

train = pd.read_excel('./price_of_books/Participants_Data/Data_Train.xlsx')

In [ ]:
kf = KFold(n_splits=3, random_state=1001,shuffle=True)
for i, (train_index, val_index) in enumerate(kf.split(train)):
    trn= train.iloc[train_index].reset_index()
    val= train.iloc[val_index].reset_index()
    
trn = trn.drop(columns=['index'])
val = val.drop(columns=['index'])

val.to_csv('sub_val.csv',index=False)
trn.to_csv('sub_train.csv',index=False)

In [2]:
from sklearn.metrics import mean_squared_log_error
import re
import gc
import matplotlib.pyplot as plt
#H2O
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator

# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
train = pd.read_csv("sub_train.csv")
test1 = pd.read_csv('sub_val.csv')

In [4]:
test = test1.drop(columns=['Price'])

In [5]:
test.head(3)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
0,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.90000,14.00000,"A layered portrait of a troubled genius for whom art was not merely a thing of beauty but a vital part of living itself.\nSelling Points: The original Marathi book won the National Award for Best Book on Cinema (in 1986), now available for the first time in English.\nThe Book: Guru Dutt is now named along with the masters of world cinema—like Orson Welles, Mizoguchi, Hitchcock, Jancso, Ophuls—for his innovative cinematic form and his deep humanism and compassion. In Guru Dutt: A Tragedy in Three Acts, renowned film-maker and scholar Arun Khopkar sheds new light on Dutt’s genius through a close examination of Dutt’s three best-known films—Pyaasa, Kaagaz Ke Phool and Sahib Biwi Aur Ghulam. With a nuanced eye, Khopkar explores the historical context which influenced Dutt’s deeply melancholic style while also analyzing the intricacies of the medium—acting, lighting, music, editing, rhythm—that Dutt carefully deployed to create his masterpieces. Originally written in Marathi, this exquisite English translation paints a layered portrait of a troubled genius for whom art was not merely a thing of beauty but a vital part of living itself.",Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts"
1,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.10000,13.00000,"A handful of grain is found in the pocket of a murdered businessman…\n\nRex Fortescue, king of a financial empire, was sipping tea in his ‘counting house’ when he suffered an agonising and sudden death. On later inspection, the pockets of the deceased were found to contain traces of cereals.\n\nYet, it was the incident in the parlour which confirmed Jane Marple’s suspicion that here she was looking at a case of crime by rhyme…",Contemporary Fiction (Books),"Crime, Thriller & Mystery"
2,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.00000,NaN,"For seven decades, ""Life"" has been thrilling the world with its unrivalled presentation of the very best photography to be found. Here, the editors have assembled the creme de la creme from the magazine's vast collection of images. Because ""Life"" has always dealt with matters of every sort, the entire spectrum of society is represented in these pages. One after another, there are unforgettable photos from Hollywood's greatest stars, from the wonders of small-town America, from the terrible wars, as well as from the zestful years of childhood. ""Life"" has always represented the apex in photojournalism and its roster of great photographers is unequalled.",Photography Textbooks,"Arts, Film & Photography"


In [6]:
h2o.init()
all_train = h2o.H2OFrame(train)
#Get columns names for Building H2O Models
target = 'Price'
predictors = [f for f in all_train.columns if f not in ['Price']]

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17" 2021-09-14; OpenJDK Runtime Environment (build 17+35-2724); OpenJDK 64-Bit Server VM (build 17+35-2724, mixed mode, sharing)
  Starting server from /root/miniconda3/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpu33kdu3f
  JVM stdout: /tmp/tmpu33kdu3f/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpu33kdu3f/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_j3mqfw
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,20 Gb
H2O_cluster_total_cores:,15
H2O_cluster_allowed_cores:,15
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
from h2o.automl import H2OAutoML

In [8]:
aml = H2OAutoML(max_models=10, seed=1331)
aml.train(x=predictors, y=target, training_frame=all_train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_1_AutoML_1_20220503_144555

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.036110338504873385
RMSE: 0.1900272046441598
MAE: 0.14506881534482094
RMSLE: 0.027596827210781032
R^2: 0.9371780276349113
Mean Residual Deviance: 0.036110338504873385
Null degrees of freedom: 4157
Residual degrees of freedom: 4151
Null deviance: 2390.0361903730927
Residual deviance: 150.14678750326354
AIC: -1993.5572547143959

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.33216282760350185
RMSE: 0.5763356900309938
MAE: 0.4228018868061516
RMSLE: 0.08161068839048603
R^2: 0.42212881849293915
Mean Residual Deviance: 0.33216282760350185
Null degrees of freedom: 4157
Residual degrees of freedom: 4151
Null deviance: 2

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.42284,0.00911,0.41121,0.42546,0.41624,0.43416,0.42713
1,mean_residual_deviance,0.33239,0.01989,0.30793,0.33401,0.31818,0.35773,0.34412
2,mse,0.33239,0.01989,0.30793,0.33401,0.31818,0.35773,0.34412
3,null_deviance,478.43753,31.12283,479.16410,508.03190,464.45612,506.80000,433.73566
4,r2,0.42084,0.02355,0.44346,0.43267,0.43658,0.40125,0.39025
5,residual_deviance,276.20935,17.67858,266.04974,287.92075,261.22592,301.56714,264.28314
6,rmse,0.57633,0.01726,0.55491,0.57794,0.56407,0.59811,0.58662
7,rmsle,0.08164,0.00216,0.07949,0.08071,0.08012,0.08365,0.08424


In [9]:
test = h2o.H2OFrame(test)
prediction = aml.leader.predict(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/root/miniconda3/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'Title' has levels not trained on: ["'One Who Will':the Search for Steve Waugh", "(ISC)2 CISSP Certified Information Systems Security Professional Official Study Guide (Isc Official Study Guides)", "10 Minutes a Day Vocabulary Ages 7-11 Key Stage 2 (Made Easy Workbooks)", "10 Practice Tests for the SAT, 2019 Edition: Extra Preparation to Help Achieve an Excellent Score (College Test Preparation)", "100 Bullets Book One", "100 Deadly Skills", "100 Lyrics", "100 Of The Most Beautiful Piano Solos Ever", "101 Classic Magic Tricks", "102 Minutes", ...1724 not listed..., "You Were My Crush", "You're Never Weird on the Internet (Almost): A Memoir", "You've Been Warned", "You: Now a Major Netflix series", "Zen And The Art Of Motorcycle Maintenance: 40th Anniversary Edition", "Zen in the Martial Arts", "Zeus Grants Stupid Wishes: A No-Bullshit Guide to World Mythology", "Zidane", "Zurich Inte

In [10]:
prediction[['predict']]

predict
5.45696
5.0163
6.68574
6.27397
6.88913
6.05948
5.6365
6.35896
6.19323
5.5384


In [11]:
pred = prediction.as_data_frame()

In [12]:
from sklearn.metrics import mean_squared_error
RMSE = np.sqrt(mean_squared_error(test1['Price'], pred))

In [13]:
RMSE

0.7045641720358767